# Teleseismic ray tracing: PKiKP

## Malcolm C. A. White
### 17 December 2020

This notebook presents a brief tutorial on how to trace raypaths for teleseismic PKiKP phases. The examples shown are 2D examples to simplify plotting and improve efficiency, but the basic procedure is identical for the 3D case.

# 1. Preliminary setup

## 1.1. Imports and constants

In [ ]:
%matplotlib ipympl

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pykonal

EARTH_RADIUS = 6371 # Earth radius in km.
CMB = 3480 # Core-mantle boundary radius in km.
ICB = 1220 # Inner-core outer-core boundary radius in km.

## 1.2. Define some convenient plotting functions

In [ ]:
def plot_field(field, ax, irho=slice(None), itheta=slice(None), iphi=slice(None), cmap=plt.get_cmap("hot"), boundaries=[ICB, CMB, EARTH_RADIUS], vmin=None, vmax=None):
    nodes = field.nodes[irho, itheta, iphi]
    xx = nodes[...,0] * np.sin(nodes[...,1]) * np.cos(nodes[...,2])
    yy = nodes[...,0] * np.sin(nodes[...,1]) * np.sin(nodes[...,2])

    qmesh = ax.pcolormesh(
        xx, 
        yy, 
        field.values[irho, itheta, iphi], 
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        shading="gouraud"
    )

    theta = np.linspace(field.min_coords[2], field.max_coords[2], 64)
    for boundary in boundaries:
        ax.plot(
            boundary * np.cos(theta),
            boundary * np.sin(theta),
            color="k",
            linewidth=1
        )

    return (qmesh)

## 1.3. Load the IASP91 velocity model

In [ ]:
iasp91 = pd.read_csv(
    "IASP91.csv",
    header=None,
    names=["depth", "radius", "Vp", "Vs"]
)

# 2. Tracing PKiKP reflections

## 2.1. Propagating the wavefront
This section consists of a loop to propagate the wavefront along different segments of the propagation path. First, the wavefront propagates from the source to the inner-core boundary. Then, the wavefront is propagated from the inner-core boundary back to the surface.

In [ ]:
# Define how many nodes to use in the radial and azimuthal directions.
nrho, nphi = 256, 256

# An empty list to hold solvers for different segments of the propagation.
solvers = []

# Define the source location.
src_loc = 5971, np.pi/2, np.pi/2

# This parameter defines segments of the propagation path. The wavefronts go
# from the source to the inner core boundary, then from the inner-core boundary
# to the Earth's surface.
path = (
    (EARTH_RADIUS, ICB), # This defines the first segment of the path.
    (ICB, EARTH_RADIUS) # This defines the second segment.
)

plt.close("all")

for ipath in range(len(path)):
    path_seg = path[ipath]

    if ipath == 0:
        # If this is the first segment of the propagation path, then use a
        # PointSourceSolver.
        solver = pykonal.solver.PointSourceSolver(coord_sys="spherical") 
    else:
        # Otherwise use an ordinary EikonalSolver.
        solver = pykonal.EikonalSolver(coord_sys="spherical")

    solvers.append(solver)

    # Define the computational domain.
    solver.vv.min_coords = min(path_seg), np.pi / 2, 0 
    solver.vv.node_intervals = (
        (max(path_seg) - min(path_seg)) / (nrho - 1), 
        1, 
        np.pi / (nphi - 1)
    )
    solver.vv.npts = nrho, 1, nphi

    # Interpolate IASP91 onto the computational grid.
    solver.vv.values = np.interp(
        solver.vv.nodes[..., 0],
        iasp91["radius"].values[-1::-1],
        iasp91["Vp"].values[-1::-1]
    )

    if ipath == 0:
        # If this is the first segment of the propagation path, set the source
        # location.
        solver.src_loc = src_loc
    else:
        # Otherwise interpolate the traveltime field of the previous segment of
        # of the propagation path onto the boundary of the current segment.
        if path_seg[0] < path_seg[1]:
            # If this is a upgoing wavefront, then interpolate onto the lower
            # boundary.
            irho = 0
        else:
            # Otherwise interpolate onto the upper boundary.
            irho = nrho - 1
        # Set the traveltime at each node along the boundary of the current
        # segment of the propagation path equal to the value at that position
        # from the previous segment.
        for iphi in range(nphi):
            idx = (irho, 0, iphi)
            node = solver.tt.nodes[idx]
            solver.tt.values[idx] = solvers[ipath-1].tt.resample(node.reshape(1, 3))
            solver.unknown[idx] = False
            solver.trial.push(*idx)

    # Finally, solve the eikonal equation for the traveltime field.
    print("Solving the eikonal equation...")
    %time solver.solve()

## 2.2. Plot the results

In [ ]:
plt.close("all")
fig, axes = plt.subplots(figsize=(12, 6), nrows=1, ncols=3)
for ax in axes:
    ax.set_aspect(1)
    
axes[0].set_title("Velocity")
qmesh = plot_field(solvers[0].vv, axes[0], itheta=0)
cbar = fig.colorbar(qmesh, ax=axes[0], orientation="horizontal")
cbar.set_label("$v_P$ (km s$^{-1}$)")
xx = src_loc[0] * np.cos(src_loc[2])
yy = src_loc[0] * np.sin(src_loc[2])
axes[0].scatter(xx, yy, marker="*", facecolor="w", edgecolor="k", linewidth=1, s=256, zorder=100)

axes[1].set_title("Downgoing wavefront")
qmesh = plot_field(solvers[0].tt, axes[1], itheta=0, cmap=plt.get_cmap("nipy_spectral_r"), vmin=0, vmax=solvers[-1].tt.values.max())
cbar = fig.colorbar(qmesh, ax=axes[1], orientation="horizontal")
cbar.set_label("Traveltime (s)")
axes[1].set_yticklabels([])

axes[2].set_title("Upgoing (reflected) wavefront")
qmesh = plot_field(solvers[1].tt, axes[2], itheta=0, cmap=plt.get_cmap("nipy_spectral_r"), vmin=0, vmax=solvers[-1].tt.values.max())
cbar = fig.colorbar(qmesh, ax=axes[2], orientation="horizontal")
cbar.set_label("Traveltime (s)")
axes[2].yaxis.tick_right()

for ax in axes:
    ax.tick_params(axis="y", left=True, right=True)

## 2.3. Trace some reflected rays

In [ ]:
rx_loc = (6371, np.pi/2, src_loc[2] - np.pi/4)

for rx_phi in src_loc[2] + np.array([7, 5, 3, 1, -2, -4, -6]) * np.pi/16:
    # Define a receiver location.
    rx_loc = (6371, np.pi/2, rx_phi)

    # Create an empty raypath.
    ray = np.empty((0, 3))

    # Trace the ray from the receiver to the inner-core-boundary.
    ipath = len(solvers) - 1 # Start with the last segment of the propagation path.
    path_seg = path[ipath]
    solver = solvers[ipath]
    ray_seg = solver.trace_ray(np.array(rx_loc))
    
    # The ray refracts along the lower (i.e., inner-core) boundary, so this is
    # a hack to drop the refracted part of the raypath.
    idx = np.where( (ray_seg[...,0] - path_seg[0]) < solver.tt.step_size * 3 )[0][-1]
    ray = np.vstack([ray, ray_seg[idx:]])

    # Trace the ray from the inner-core boundary to the source.
    ipath -= 1
    solver = solvers[ipath]
    ray_seg = solver.trace_ray(ray[0])
    ray = np.vstack([ray_seg, ray])
    
    
    # Plot the raypath on the velocity model.
    xx = ray[..., 0] * np.cos(ray[..., 2])
    yy = ray[..., 0] * np.sin(ray[..., 2])
    axes[0].plot(xx, yy, color="k", linewidth=0.5)